In [114]:

from itertools import combinations_with_replacement
from collections import defaultdict
from itertools import product, combinations

def Sup(x):
    """
    x is a list or tuple of length n representing (x_n, ..., x_1)
    returns Sup(x) = (k_l, ..., k_1)
    """
    n = len(x)
    support = []

    for i in range(n):
        if x[n - 1 - i] != 0:
            support.append(i + 1)

    # if not support:
    #     return (0,)
    return tuple(sorted(support, reverse=True))

def compute_H_eps(x_vec, k_list):
    """"
    out a dictionary for H0 = 0, H1, H2, ...
    """
    H_eps = {0: 0}
    cumulative = 0

    for eps in range(1, len(k_list) + 1):
        k_eps = k_list[-eps]
        cumulative += x_vec[len(x_vec) - k_eps]
        H_eps[eps] = cumulative

    return H_eps


def prepare_all_memory(n, d):
    arrays = dict()
    summary = defaultdict(list)

    # -------------------------------
    # Type-0
    # -------------------------------
    arrays[(0,)] = [0] * (n + 2)
    summary[0].append(n + 2)

    # -------------------------------
    # Type-1 to Type-(d-1)
    # -------------------------------
    for i in range(1, d):
        for combo in combinations_with_replacement(range(1, n + 1), i):
            label = tuple(sorted(combo, reverse=True))
            k_t = label[0]
            length = n + 2 - k_t
            arrays[label] = [0] * length
            summary[i].append(length)

    # -------------------------------
    # Type-d
    # -------------------------------
    for combo in combinations_with_replacement(range(1, n + 1), d):
        label = tuple(sorted(combo, reverse=True))
        arrays[label] = [0]
        summary[d].append(1)

    return arrays, summary

def compute_all_derivatives_at_0(arrays, f, n, q):
    """
    This is initialization phase, where tha value of all partial deriavtives 
    at 0 is obtained.
    """
    derivatives = {}

    for label in arrays:

        if label == (0,):
            derivatives[label] = f([0] * n) % q
            continue

        t = len(label)
        value = 0

        for r in range(t + 1):
            for idxs in combinations(range(t), r):
                x = [0] * n
                for i in idxs:
                    x[label[i] - 1] += 1   # x = (x1,...,xn)

                sign = (-1) ** (t - r)
                value += sign * f(list(reversed(x)))

        derivatives[label] = value % q

    return derivatives

def build_label(k_list, x_vec, eps, delta):
    """
    Build label:
    (k_0^{x_{k_0}}, ..., k_{eps-1}^{x_{k_{eps-1}}}, k_eps^{delta})

    k_list : (k_l, ..., k_1)
    eps    : integer ε (1 ≤ ε ≤ l)
    x_vec  : (x_n, ..., x_1)
    """

    if eps < 0 or eps > len(k_list):
        raise ValueError("eps must satisfy 1 ≤ eps ≤ len(k_list)")

    label = []
    if eps > 1:
        for k in reversed(k_list[-(eps - 1):]):
            power = x_vec[len(x_vec) - k]
            label.extend([k] * power)
    if eps>0:
        k_eps = k_list[-eps]
        label.extend([k_eps] * delta)

    return tuple(sorted(label, reverse=True)) if label else (0,)


def EVA(arrays, x_vec, H, H_eps, d, k_list, q):
    """
    arrays : dict from prepare_all_memory
    x_vec  : (x_n, ..., x_1) in F_q
    H      : Hw(x)
    H_eps  : dict with H_eps[0] = 0
    d      : degree
    k_list : (k_l, ..., k_1,0)
    q      : field size
    """
    count_ops=0
    l = len(k_list)
    # --------------------------------------------------
    # Case H = 1
    # --------------------------------------------------
    if H == 1:
        # print("H=1, only single step calculation")
        A0 = arrays[(0,)]
        
        k1 = k_list[-1]
        
        A0[k1 + 1] = (A0[1] + arrays[(k1,)][0]) % q
        count_ops+=1
        # print(A0, k1,arrays[(k1,)][0])
        return A0[k1 + 1], arrays, count_ops
    if d == 1:
        # print("here, d=1, again single step calculation ")
        A0 = arrays[(0,)]
        
        k1 = k_list[-1]
        x_k1=x_vec[len(x_vec)-k1]
        if x_k1>1:
            A0[k1 + 1] = (A0[k1+1] + arrays[(k1,)][0]) % q
            count_ops+=1
        else:
            k2 = k_list[-2]
            A0[k1 + 1] = (A0[k2+1] + arrays[(k1,)][0]) % q
            count_ops+=1
        return A0[k1 + 1], arrays, count_ops
    # --------------------------------------------------
    # Main loops
    # --------------------------------------------------
    for eps_idx in range(l, 0, -1):
        ε = eps_idx
        k_eps = k_list[l-eps_idx]
        η = H_eps[ε - 1]
        x_keps = x_vec[len(x_vec) - k_eps]
        for δ in range(x_keps - 1, -1, -1):
            step = η + δ
            if step >= d:
                continue

            # --------------------------------------------------
            # Case step = H - 1
            # --------------------------------------------------
            if step == H-1 :

                if x_keps > 1:
                    label = build_label(k_list, x_vec, eps_idx, delta=δ)
                    label_next = build_label(k_list, x_vec, eps_idx, delta=δ + 1)
                    
                    arrays[label][1] = (arrays[label][0] + arrays[label_next][0]) % q
                    count_ops+=1
                    
                else:
                    label1 = build_label(k_list, x_vec, eps_idx, delta=0 )
                    label2 = build_label(k_list, x_vec, eps_idx, delta=1)
                    num1 = 0 if eps_idx==1 else 1
                    
                    idx = k_eps - k_list[(l-eps_idx+1)*num1]*num1 + 1

                    arrays[label1][idx] = (arrays[label1][0] + arrays[label2][0]) % q
                    count_ops+=1
            # --------------------------------------------------
            # Case step <= H - 2
            # --------------------------------------------------
            elif step <= H - 2:
                num = 0 if step == d - 1 else 1
                # print("going to multiply by num=", num)
                # ------------------------------
                # δ > 0
                # ------------------------------
                if δ > 0:
                    a = x_keps - δ
                    label = build_label(k_list, x_vec,  eps_idx, delta=δ)
                    label_next = build_label(k_list, x_vec,  eps_idx, delta=δ + 1)
                    if a > 1:
                        arrays[label][1] = (arrays[label][1] + arrays[label_next][1 * num]) % q                        
                        count_ops+=1
                    else:
                        idx = k_list[l-eps_idx - 1] - k_eps + 1
                        arrays[label][1] = (arrays[label][idx]+ arrays[label_next][idx * num]) % q
                        count_ops+=1
                # ------------------------------
                # δ = 0
                # ------------------------------
                else:
                    a = x_keps
                    label1 = build_label(k_list, x_vec, eps_idx, delta=0 )
                    label2 = build_label(k_list, x_vec, eps_idx, delta=1)
                    num1 = 0 if eps_idx==1 else 1
                    idx = k_eps - k_list[(l-eps_idx+1)*num1]*num1 + 1
                    if a > 1:
                        arrays[label1][idx] = (arrays[label1][idx]+ arrays[label2][1 * num]) % q
                        count_ops+=1
                    else:
                        num1 = 0 if eps_idx==1 else 1
                        idx1 = k_list[l-eps_idx-1]  - k_list[(l-eps_idx+1)*num1]*num1  + 1
                        arrays[label1][idx] = (arrays[label1][idx1]+ arrays[label2][(idx1-idx+1) * num]) % q
                        count_ops+=1
    # --------------------------------------------------
    # Return
    # --------------------------------------------------
    return arrays[(0,)][k_list[-1] + 1] % q, arrays,count_ops
import math
def comb(n, k):
    return math.factorial(n) // (math.factorial(k) * math.factorial(n-k))
def in_partial_space_type1(x, parts):
    idx = 0
    for n_i, w_i in parts:
        block = x[idx:idx + n_i]
        if sum(v for v in block if v != 0) > w_i:
            return False
        idx += n_i
    return True
def in_partial_space_type2(x, parts):
    idx = 0
    for n_i, w_i in parts:
        block = x[idx:idx + n_i]
        if sum(1 for v in block if v != 0) > w_i:
            return False
        idx += n_i
    return True



In [100]:
import re

def build_polynomial(poly_str, n, q):
    """
    poly_str : string like "2*x1*x2 + 3*x1 + 4*x3 + 3"
    n        : number of variables
    q        : field size

    returns:
        f(x)    : callable, x = (x_n, ..., x_1)
        degree  : total degree of polynomial
    """
    poly_str = poly_str.replace(" ", "")

    # Compute degree
    # def monomial_degree(term):
    #     vars = re.findall(r"x\d+", term)
    #     return len(vars)
    def monomial_degree(term):
        degree = 0
        for var, power in re.findall(r"(x\d+)(?:\*\*(\d+))?", term):
            degree += int(power) if power else 1
        return degree

    terms = poly_str.replace("-", "+-").split("+")
    degree = max(monomial_degree(term) for term in terms if term)

    # Build evaluator
    def f(x):
        """
        x is given as (x_n, ..., x_1)
        """
        vals = {f"x{i+1}": x[n - i - 1] for i in range(n)}
        return eval(poly_str, {}, vals) % q

    return f, degree

In [116]:
from itertools import product

n = 3
q = 5
# d = 4
parts = [
    (1,1),   # P^{w1}_{n1}
    (1,1),   # P^{w2}_{n2}
    (1, 0)    # P^{w3}_{n3}
]
assert sum(n_i for n_i, _ in parts) == n

poly="2*x1*x2*x2*x3*x3+3*x1*x1*x2+4*x3*x1+4*x2*x2*x2*x3"
f, d = build_polynomial(poly, n, q)
print("Variables(n):", n, ", Degree(d):", d, ", Order of field(q):",q, "\n","polynomial:", poly)
print(f"|F_q^n| (full space size) : {q**n}")
# --------------------------------------------------
# Step 1: Prepare memory ONCE
# --------------------------------------------------
arrays, _ = prepare_all_memory(n, d)


# --------------------------------------------------
# Step 2: Initialize array[label][0] for ALL labels
# ---------------------------------------------
derivatives_at_0 = compute_all_derivatives_at_0(arrays, f, n, q)
 
for label, value in derivatives_at_0.items():
    if label not in arrays:
        raise KeyError(f"Label {label} not found in arrays")

    if label == (0,):
        arrays[label][1] = value   # special rule for (0,)
    else:
        arrays[label][0] = value
# --------------------------------------------------
# Step 3: Iterate over F_q^n in lex order
# --------------------------------------------------
results = {}
newarrays=arrays
t_oprs=0
card_part_space=0
for x in product(range(q), repeat=n):
    if not in_partial_space_type2(x, parts):
        continue
    card_part_space+=1
    if all(v == 0 for v in x):
        results[tuple(list(x))] = newarrays[(0,)][1]
        continue
    x_vec = list(x)              # (x_n, ..., x_1)
    k_list = Sup(x_vec)
    H = sum(x_vec)        
    H_eps = compute_H_eps(x_vec, k_list)
    val, newarrays, oprs = EVA(
        arrays=newarrays,
        x_vec=x_vec,
        H=H,
        H_eps=H_eps,
        d=d,
        k_list=k_list,
        q=q
    )
    
    t_oprs+=oprs
    results[tuple(x_vec)] = val
print("Evaluation part done")
print("\nStructured partial space parameters:")
print("  Blocks (n_i, w_i):", parts)

print("\nTime Complexity Check:")
print(f"  |S| (partial space size)  : {card_part_space}")
print(f"  Total '+' operations      : {t_oprs}")
print(f"  d · |S|                   : {d * card_part_space}")

alloted_mem = sum(len(v) for v in arrays.values())

print("\nMemory Complexity Check:")
print(f"  Allocated memory  : arrays A_(.)    ={alloted_mem - 1}")
print(f"  Theoretical bound : 2·C(n+d, d) − 1 ={2 * comb(n+d, d) - 1}")

print("\nFunctional Correctness Check:")
poly_results = {x: f(x) for x in product(range(q), repeat=n)}

for x in product(range(q), repeat=n):
    if not in_partial_space_type2(x, parts):
        continue

    eva_val  = results[x]
    poly_val = poly_results[x]

    if eva_val != poly_val:
        print("  ❌ Mismatch detected at x =", x)
        print(f"     EVA(x)  = {eva_val}")
        print(f"     f(x)    = {poly_val}")
        break
else:
    print("✅ All values match")


Variables(n): 3 , Degree(d): 5 , Order of field(q): 5 
 polynomial: 2*x1*x2*x2*x3*x3+3*x1*x1*x2+4*x3*x1+4*x2*x2*x2*x3
|F_q^n| (full space size) : 125
Evaluation part done

Structured partial space parameters:
  Blocks (n_i, w_i): [(1, 1), (1, 1), (1, 0)]

Time Complexity Check:
  |S| (partial space size)  : 25
  Total '+' operations      : 90
  d · |S|                   : 125

Memory Complexity Check:
  Allocated memory  : arrays A_(.)    =111
  Theoretical bound : 2·C(n+d, d) − 1 =111

Functional Correctness Check:
✅ All values match


In [101]:
"""
Optional: If want to generate a random poly and check
"""
def random_polynomial(n, q, max_degree, sparsity=1.0):
    """
    n           : number of variables
    q           : field size
    max_degree  : maximum total degree (must be ≥ 1)
    sparsity    : probability that a monomial is included (0 < sparsity ≤ 1)

    returns:
        poly_str : non-constant polynomial
    """
    import itertools, random

    if max_degree < 1:
        raise ValueError("max_degree must be at least 1 to generate a non-constant polynomial")

    terms = []

    for deg in range(max_degree + 1):
        for exps in itertools.product(range(deg + 1), repeat=n):
            if sum(exps) != deg:
                continue

            # random sparsity filter
            if random.random() > sparsity:
                continue

            coeff = random.randrange(q)
            if coeff == 0:
                continue

            monomial = []
            for i, e in enumerate(exps):
                if e == 1:
                    monomial.append(f"x{i+1}")
                elif e > 1:
                    monomial.append(f"x{i+1}**{e}")

            if monomial:  # degree ≥ 1 term
                term = f"{coeff}*" + "*".join(monomial)
                terms.append(term)

    # ------------------------------------------------
    # Guarantee non-constant polynomial
    # ------------------------------------------------
    if not terms:
        # force a random linear term
        i = random.randrange(n)
        coeff = random.randrange(1, q)
        terms.append(f"{coeff}*x{i+1}")

    return " + ".join(terms)


In [118]:
from itertools import product
n =3
q = 5
max_degree =5
# example: n = n1 + n2 + n3
parts = [
    (1,1),   # P^{w1}_{n1}
    (1,1),   # P^{w2}_{n2}
    (1, 0)    # P^{w3}_{n3}
]
assert sum(n_i for n_i, _ in parts) == n
poly = random_polynomial(n, q, max_degree, sparsity=0.5)
f, d = build_polynomial(poly, n, q)
print("Variables(n):", n, ", Degree(d):", d, ", Order of field(q):",q, "\n","polynomial:", poly)
print(f"|F_q^n| (full space size) : {q**n}")
arrays, _ = prepare_all_memory(n, d)
derivatives_at_0 = compute_all_derivatives_at_0(arrays, f, n, q)

# print(arrays)
for label, value in derivatives_at_0.items():
    if label not in arrays:
        raise KeyError(f"Label {label} not found in arrays")

    if label == (0,):
        arrays[label][1] = value   # special rule for (0,)
    else:
        arrays[label][0] = value
results = {}
newarrays=arrays
t_oprs=0
card_part_space=0
for x in product(range(q), repeat=n):
    if not in_partial_space_type2(x, parts):
        continue
    card_part_space+=1
    if all(v == 0 for v in x):
        results[tuple(list(x))] = newarrays[(0,)][1]
        continue
    x_vec = list(x)              # (x_n, ..., x_1)
    k_list = Sup(x_vec)
    H = sum(x_vec)        
    H_eps = compute_H_eps(x_vec, k_list)
    val, newarrays, oprs = EVA(
        arrays=newarrays,
        x_vec=x_vec,
        H=H,
        H_eps=H_eps,
        d=d,
        k_list=k_list,
        q=q
    )
    
    t_oprs+=oprs
    results[tuple(x_vec)] = val
print("Evaluation part done")
print("\nStructured partial space parameters:")
print("  Blocks (n_i, w_i):", parts)

print("\nTime Complexity Check:")
print(f"  |S| (partial space size)  : {card_part_space}")
print(f"  Total '+' operations      : {t_oprs}")
print(f"  d · |S|                   : {d * card_part_space}")

alloted_mem = sum(len(v) for v in arrays.values())

print("\nMemory Complexity Check:")
print(f"  Allocated memory  : arrays A_(.)    ={alloted_mem - 1}")
print(f"  Theoretical bound : 2·C(n+d, d) − 1 ={2 * comb(n+d, d) - 1}")

print("\nFunctional Correctness Check:")
poly_results = {x: f(x) for x in product(range(q), repeat=n)}

for x in product(range(q), repeat=n):
    if not in_partial_space_type2(x, parts):
        continue

    eva_val  = results[x]
    poly_val = poly_results[x]

    if eva_val != poly_val:
        print("  ❌ Mismatch detected at x =", x)
        print(f"     EVA(x)  = {eva_val}")
        print(f"     f(x)    = {poly_val}")
        break
else:
    print("✅ All values match")



Variables(n): 3 , Degree(d): 5 , Order of field(q): 5 
 polynomial: 3*x1 + 2*x2*x3 + 1*x2**2 + 2*x3**3 + 1*x2**2*x3 + 2*x1*x2*x3 + 3*x2**2*x3**2 + 3*x2**3*x3 + 2*x2**4 + 4*x1**3*x2 + 3*x2*x3**4 + 1*x2**2*x3**3 + 2*x1*x2**2*x3**2 + 2*x1*x2**3*x3 + 3*x1*x2**4 + 2*x1**2*x2**2*x3 + 1*x1**2*x2**3 + 2*x1**4*x3 + 4*x1**4*x2
|F_q^n| (full space size) : 125
Evaluation part done

Structured partial space parameters:
  Blocks (n_i, w_i): [(1, 1), (1, 1), (1, 0)]

Time Complexity Check:
  |S| (partial space size)  : 25
  Total '+' operations      : 90
  d · |S|                   : 125

Memory Complexity Check:
  Allocated memory  : arrays A_(.)    =111
  Theoretical bound : 2·C(n+d, d) − 1 =111

Functional Correctness Check:
✅ All values match


In [41]:

D = {}

def recurse(label,D):
    D[label] = None   #A[label]= D_label(0)

    # stop condition
    if len(label) == d + 1:
        return

    # decide starting index
    if len(label) == 1:      # label == (0,)
        start = 1
    else:
        start = label[-1]
    # print(f"start={start} to end={n+1}")
    for j in range(start, n + 1):
        recurse(label + (j,), D)
        print(D)

print(recurse((0,),D={}))


{(0,): None, (0, 1): None, (0, 1, 1): None}
{(0,): None, (0, 1): None, (0, 1, 1): None, (0, 1, 2): None}
{(0,): None, (0, 1): None, (0, 1, 1): None, (0, 1, 2): None, (0, 1, 3): None}
{(0,): None, (0, 1): None, (0, 1, 1): None, (0, 1, 2): None, (0, 1, 3): None}
{(0,): None, (0, 1): None, (0, 1, 1): None, (0, 1, 2): None, (0, 1, 3): None, (0, 2): None, (0, 2, 2): None}
{(0,): None, (0, 1): None, (0, 1, 1): None, (0, 1, 2): None, (0, 1, 3): None, (0, 2): None, (0, 2, 2): None, (0, 2, 3): None}
{(0,): None, (0, 1): None, (0, 1, 1): None, (0, 1, 2): None, (0, 1, 3): None, (0, 2): None, (0, 2, 2): None, (0, 2, 3): None}
{(0,): None, (0, 1): None, (0, 1, 1): None, (0, 1, 2): None, (0, 1, 3): None, (0, 2): None, (0, 2, 2): None, (0, 2, 3): None, (0, 3): None, (0, 3, 3): None}
{(0,): None, (0, 1): None, (0, 1, 1): None, (0, 1, 2): None, (0, 1, 3): None, (0, 2): None, (0, 2, 2): None, (0, 2, 3): None, (0, 3): None, (0, 3, 3): None}
None


In [ ]:
def build_tree(n, d):
    D = {}

    def recurse(label):
        D[label] = None   #A[label]= D_label(0)

        # stop condition
        if len(label) == d + 1:
            return

        # decide starting index
        if len(label) == 1:      # label == (0,)
            start = 1
        else:
            start = label[-1]
        # print(f"start={start} to end={n+1}")
        for j in range(start, n + 1):
            recurse(label + (j,))
            # print(D)

    recurse((0,))
    return D


n = 3
q = 5
# max_degree =2
# poly = random_polynomial(n, q, max_degree, sparsity=0.5)
# f, d = build_polynomial(poly, n, q)
# print("# variables: ", n, "Degree:", d,"\n","polynomial:", poly)
# arrays, _ = prepare_all_memory(n, d)
# derivatives_at_0 = compute_all_derivatives_at_0(arrays, f, n, q)
# for label, value in derivatives_at_0.items():
#     if label not in arrays:
#         raise KeyError(f"Label {label} not found in arrays")
#     print((0,)+label)
#     if label == (0,):
#         arrays[label][1] = value   # special rule for (0,)
#     else:
#         arrays[label][0] = value
D = build_tree(n=n, d=d)
for k in D:
    print(k)


(0,)
(0, 1)
(0, 1, 1)
(0, 1, 2)
(0, 1, 3)
(0, 2)
(0, 2, 2)
(0, 2, 3)
(0, 3)
(0, 3, 3)


In [ ]:
# Enumerate (k_h, ..., k_1) for inputs in P_{n_s,...,n_1}^{w_s,...,w_1}

def ENU(K, H, I, W, N, h, end):
    """
    Recursive enumeration procedure.
    
    K : array, where K[h-i+1] stores k_i
    H : array, H[num] stores partial Hamming weights
    I : array, index-to-block mapping
    W : array, weight constraints (w_1,...,w_s)
    N : array, cumulative block sizes
    h : current Hamming weight
    end : upper bound for index enumeration
    """

    for i in range(1, end + 1):
        # K[h+1] = i
        K[h + 1] = i
        num = I[i]

        # increase the Hamming weight
        H[num] += 1

        # Output h+1 and the necessary (at most d+1) indices from K
        # (Here we output (k_{h+1}, ..., k_1))
        print("h =", h + 1, ", support =", K[1:h + 2])

        if H[num] < W[num]:
            ENU(K, H, I, W, N, h + 1, K[h + 1] - 1)

        elif H[num] == W[num] and num > 0:
            ENU(K, H, I, W, N, h + 1, N[num - 1])

        else:
            # reset the Hamming weight
            H[num] -= 1

    # backtracking
    if h > 0:
        num = I[K[h]]
        H[num] -= 1


def MAIN():
    """
    Main procedure
    """

    # Example initialization (replace with actual parameters)
    # --------------------------------------------------------
    # w_1,...,w_s
    W = [None, 2, 1]      # W[1]=w1, W[2]=w2 (1-based indexing)

    # n_1,...,n_s
    n_blocks = [2, 3]    # n1=2, n2=3
    s = len(n_blocks)

    # N[] = {n1, n1+n2, ..., sum n_i}
    N = [0]
    for ni in n_blocks:
        N.append(N[-1] + ni)

    n = N[-1]

    # K[n+1] = {n+1,0,...,0}
    K = [0] * (n + 2)
    K[0] = n + 1

    # H[s] = {0,...,0}
    H = [0] * (s + 1)

    # Initialize I[]
    # I[0] = 0
    # I[i] = j if N[j-1] < i <= N[j]
    I = [0] * (n + 1)
    I[0] = 0
    for i in range(1, n + 1):
        for j in range(1, s + 1):
            if N[j - 1] < i <= N[j]:
                I[i] = j
                break

    # Call ENU
    ENU(K, H, I, W, N, 0, n)


# Run MAIN
MAIN()
print()

h = 1 , support = [1]
h = 1 , support = [2]
h = 2 , support = [2, 1]
h = 1 , support = [3]
h = 2 , support = [3, 1]
h = 2 , support = [3, 2]
h = 3 , support = [3, 2, 1]
h = 1 , support = [4]
h = 2 , support = [4, 1]
h = 2 , support = [4, 2]
h = 3 , support = [4, 2, 1]
h = 1 , support = [5]
h = 2 , support = [5, 1]
h = 2 , support = [5, 2]
h = 3 , support = [5, 2, 1]
